In [1]:
import os
import glob
import pandas as pd
import numpy as np

# 기본 경로 및 폴더 순서 정의
base_path = "../raw_data/투자지표/"
folders_to_process = ['나현', '수정', '유진', '의찬', '지윤']

# 모든 처리된 데이터프레임을 저장할 빈 리스트 생성
all_dfs_list = []

# 지정된 폴더 순서대로 반복
for folder_name in folders_to_process:
    # 현재 폴더의 전체 경로 생성
    current_folder_path = os.path.join(base_path, folder_name)

    # 현재 폴더 내의 모든 .xlsx 파일 목록 가져오기
    file_paths = glob.glob(os.path.join(current_folder_path, "*.xlsx"))

    print(f"--- '{folder_name}' 폴더 처리 중... (파일 {len(file_paths)}개) ---")

    # 해당 폴더 내의 각 파일에 대해 반복
    for file_path in file_paths:

        # 1. 데이터 로드 (첫 번째 열을 인덱스로)
        df = pd.read_excel(file_path, index_col=0)

        # 2. '기업' 이름 추출 (예: 'ABT')
        company_name = os.path.splitext(os.path.basename(file_path))[0]

        # 3. DataFrame 전치 (행/열 뒤집기)
        df_T = df.T

        # 4. 인덱스(날짜)를 '날짜' 컬럼으로 리셋
        df_processed = df_T.reset_index(names='날짜')

        # 5. '년도'와 '분기' 컬럼 생성
        date_parts = df_processed['날짜'].str.split('.')
        # 달력 기준 년도와 월을 숫자로 먼저 추출
        calendar_year = ('20' + date_parts.str[0]).astype(int)
        month_as_int = date_parts.str[1].astype(int)

        # 회계 연도(FY) 계산
        # 10월(10)보다 큰 달(10, 11, 12월)은 달력 연도 + 1이 회계 연도임
        # 예: 2024년 10월 -> 2025 회계연도
        fiscal_year = np.where(month_as_int > 9, calendar_year + 1, calendar_year)

        # 회계 분기(FY Quarter) 계산 (위와 동일한 로직)
        fiscal_month = (month_as_int + 2) % 12 + 1
        quarter_number = (fiscal_month - 1) // 3 + 1

        # 데이터프레임에 할당
        df_processed['회계연도'] = fiscal_year.astype(str) # '년도' 대신 '회계연도'로 이름 변경 권장
        df_processed['분기'] = 'FY' + quarter_number.astype(str) + 'Q'

        # 6. '기업' 컬럼 추가
        df_processed['기업'] = company_name

        # 7. 년도 필터링 (2022년 이후)
        df_filtered = df_processed[df_processed['회계연도'] >= '2022'].copy()

        # 8. 최종 컬럼 순서 정리
        final_columns = ['기업', '날짜', '회계연도', '분기'] + df.index.tolist()
        df_final_single = df_filtered[final_columns]

        # 9. 처리 완료된 단일 데이터프레임을 리스트에 추가
        all_dfs_list.append(df_final_single)

    print(f"--- '{folder_name}' 폴더 처리 완료 ---")

# 리스트에 저장된 모든 데이터프레임을 하나로 합치기
final_combined_df = pd.concat(all_dfs_list, ignore_index=True)

# 결측값(NaN)을 -으로 채우기
final_combined_df = final_combined_df.fillna('-')

# 최종 결과 확인
print("\n========================================")
print("모든 파일이 성공적으로 합쳐졌습니다.")
print(f"최종 DataFrame 크기: {final_combined_df.shape}")
print("========================================")
final_combined_df.head()

--- '나현' 폴더 처리 중... (파일 12개) ---
--- '나현' 폴더 처리 완료 ---
--- '수정' 폴더 처리 중... (파일 12개) ---
--- '수정' 폴더 처리 완료 ---
--- '유진' 폴더 처리 중... (파일 12개) ---
--- '유진' 폴더 처리 완료 ---
--- '의찬' 폴더 처리 중... (파일 12개) ---
--- '의찬' 폴더 처리 완료 ---
--- '지윤' 폴더 처리 중... (파일 11개) ---
--- '지윤' 폴더 처리 완료 ---

모든 파일이 성공적으로 합쳐졌습니다.
최종 DataFrame 크기: (921, 19)


항목,기업,날짜,회계연도,분기,주당순이익\nEPS(달러),주당순자산\nBPS(달러),주당매출액\nSPS(달러),주당현금흐름\nCPS(달러),주당배당금\nDPS(달러),주가수익배수\nPER(배),주가순자산배수\nPBR(배),주가매출액배수\nPSR(배),주가현금흐름\n배수 PCR(배),자기자본이익률\nROE(%),총자산이익률\nROA(%),투하자본이익률\nROIC(%),부채비율(%),유동비율(%),이자보상\n배수(배)
0,ABT,25.09.30,2025,FY4Q,7.97,29.43,25.17,5.23,2.32,16.68,4.55,5.32,25.56,28.1,16.9,19.0,64.2,170.3,15.0
1,ABT,25.06.30,2025,FY3Q,7.97,29.15,24.73,5.18,2.28,16.92,4.65,5.49,26.17,29.8,17.4,19.5,65.3,182.1,14.0
2,ABT,25.03.31,2025,FY2Q,7.70,28.21,24.35,5.15,2.24,17.04,4.69,5.43,25.70,30.6,17.4,19.6,66.0,178.0,13.0
3,ABT,24.12.31,2025,FY1Q,7.64,27.53,24.11,4.92,2.20,14.64,4.1,4.68,22.92,32.2,17.8,20.1,70.0,167.1,12.0
4,ABT,24.09.30,2024,FY4Q,3.29,23.01,23.70,5.02,2.16,34.40,4.96,4.81,22.73,14.7,7.9,20.9,85.8,159.7,12.0


In [2]:
# 저장할 경로와 파일명 지정
save_path = "../raw_data/투자지표-ver.csv"

# DataFrame을 CSV 파일로 저장
final_combined_df.to_csv(save_path, index=False, encoding='utf-8-sig')